In [3]:
from PIL import Image

width = 100
height = 100

# l - znaczy ze w skali szarosci
image = Image.new("L", (width, height), color=255)

# dodanie czarnego paska
for x in range(width):
    for y in range(height):
        if width // 3 <= x < 2 * (width // 3):
            image.putpixel((x, y), 0)  

image.save("sample_image.png")
image.show()

In [ ]:
from PIL import Image
import random

def encrypt_image(image_path):
    original_image = Image.open(image_path)
    
    share1 = Image.new("L", (original_image.width, original_image.height))
    share2 = Image.new("L", (original_image.width, original_image.height))
    
    
    for x in range(original_image.width):
        for y in range(original_image.height):
            pixel = original_image.getpixel((x, y))
            # pobranie randomowych udzialow
            random_share1 = random.randint(0, 255)
            # wartosc pixela z org obrazu - random_share1 - modulo zeby w przedziale
            random_share2 = (pixel - random_share1) % 256
            share1.putpixel((x, y), random_share1)
            share2.putpixel((x, y), random_share2)
    
    return share1, share2

def decrypt_image(share1, share2):
    decrypted_image = Image.new("L", (share1.width, share1.height))
    
    for x in range(share1.width):
        for y in range(share1.height):
            pixel1 = share1.getpixel((x, y))
            pixel2 = share2.getpixel((x, y))
            # laczenie
            decrypted_pixel = (pixel1 + pixel2) % 256
            decrypted_image.putpixel((x, y), decrypted_pixel)
    
    return decrypted_image

In [4]:
image_path = "sample_image.png"

share1, share2 = encrypt_image(image_path)

share1.save("share1.png")
share2.save("share2.png")

decrypted_image = decrypt_image(share1, share2)
decrypted_image.save("decrypted_image.png")
